<a href="https://colab.research.google.com/github/MrvZzZzZ/Lab1_AI_for_study/blob/main/model_and_checks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gdown

In [34]:
!gdown "https://drive.google.com/uc?id=1ox9otamMD6Llut4RENR7agrWPn9_cX60" -O data.csv

Downloading...
From: https://drive.google.com/uc?id=1ox9otamMD6Llut4RENR7agrWPn9_cX60
To: /content/data.csv
100% 34.0M/34.0M [00:00<00:00, 145MB/s]


In [35]:
!gdown "https://drive.google.com/uc?id=19Zig529RJktMXmiUUczN2k2o4PaRa6gu" -O data2.csv

Downloading...
From: https://drive.google.com/uc?id=19Zig529RJktMXmiUUczN2k2o4PaRa6gu
To: /content/data2.csv
100% 30.4M/30.4M [00:00<00:00, 86.6MB/s]


In [36]:
import pandas as pd
import ast

df = pd.read_csv("./data.csv")
df2 = pd.read_csv("./data2.csv")

df['embedding'] = df['embedding'].apply(ast.literal_eval)
df2['embedding'] = df2['embedding'].apply(ast.literal_eval)

df = pd.concat([df, df2], ignore_index=True)

In [20]:
df['text'].unique()


array(["Операция «Монастырь»\n\n{{другие значения|Монастырь (значения)}}\n\n{{Вооружённый конфликт\n |конфликт = Операция советской разведки «Монастырь»\n |часть = [[Вторая мировая война]]\n |дата = 1941—1944\n |место = [[Москва]] \n |итог = Массированная дезинформация германского командования и нейтрализация диверсионных групп \n |противник2 = {{Флагификация|Третий рейх}}\n |противник1 = {{Флагификация|СССР}}\n |командир1 = {{Флаг СССР}} [[Судоплатов, Павел Анатольевич|Павел Судоплатов]] \n |командир2 = {{Флагификация|Третий рейх}} Руководство Абвера \n |силы1 = {{Флагификация|СССР}} [[НКВД]]\n |силы2 = {{Флагификация|Третий рейх}} [[Абвер]]     \n}}\n\n'''Операция «Монастырь»'''\xa0— одна из самых успешных операций советских спецслужб во время [[Великая Отечественная война|Великой Отечественной войны]]. Основана на [[радиоигра|радиоигре]], которую группа оперативных работников [[Народный комиссариат внутренних дел СССР|НКВД]] вела со структурами [[Абвер]]а, и продолжалась с 1941 по 1

In [4]:
!pip install openai

In [37]:
from scipy import spatial
from openai import OpenAI
import os
import tiktoken
import pandas as pd
from google.colab import userdata
EMBEDDING_MODEL = "text-embedding-ada-002"
GPT_MODEL = "gpt-3.5-turbo"

key = userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = key
openai = OpenAI(
    base_url="https://api.vsegpt.ru/v1",
    api_key=userdata.get("OPENAI_API_KEY")
)

# Функция поиска
def strings_ranked_by_relatedness(
    query: str, # пользовательский запрос
    df: pd.DataFrame, # DataFrame со столбцами text и embedding (база знаний)
    relatedness_fn=lambda x, y: 1 - spatial.distance.cosine(x, y), # функция схожести, косинусное расстояние
    top_n: int = 100 # выбор лучших n-результатов
) -> tuple[list[str], list[float]]: # Функция возвращает кортеж двух списков, первый содержит строки, второй - числа с плавающей запятой
    """Возвращает строки и схожести, отсортированные от большего к меньшему"""

    # Отправляем в OpenAI API пользовательский запрос для токенизации
    query_embedding_response = openai.embeddings.create(
        model=EMBEDDING_MODEL,
        input=query,
    )

    # Получен токенизированный пользовательский запрос
    query_embedding = query_embedding_response.data[0].embedding

    # Сравниваем пользовательский запрос с каждой токенизированной строкой DataFrame
    strings_and_relatednesses = [
        (row["text"], relatedness_fn(query_embedding, row["embedding"]))
        for i, row in df.iterrows()
    ]

    # Сортируем по убыванию схожести полученный список
    strings_and_relatednesses.sort(key=lambda x: x[1], reverse=True)

    # Преобразовываем наш список в кортеж из списков
    strings, relatednesses = zip(*strings_and_relatednesses)

    # Возвращаем n лучших результатов
    return strings[:top_n], relatednesses[:top_n]

# с этой функцией мы уже знакомы
def num_tokens(text: str, model: str = GPT_MODEL) -> int:
    """Возвращает число токенов в строке для заданной модели"""
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))

# Функция формирования запроса к chatGPT по пользовательскому вопросу и базе знаний
def query_message(
    query: str, # пользовательский запрос
    df: pd.DataFrame, # DataFrame со столбцами text и embedding (база знаний)
    model: str, # модель
    token_budget: int # ограничение на число отсылаемых токенов в модель
) -> str:
    """Возвращает сообщение для GPT с соответствующими исходными текстами, извлеченными из фрейма данных (базы знаний)."""
    strings, relatednesses = strings_ranked_by_relatedness(query, df) # функция ранжирования базы знаний по пользовательскому запросу
    # Шаблон инструкции для chatGPT
    message = (
    'Используйте приведённые ниже статьи о Великой Отечественной войне, чтобы дать **точный и краткий** ответ. '
    'Если в статьях есть конкретная дата, имя, число или название — назови его дословно. '
    'Если информации недостаточно — ответь только: "Я не помню."\n\n'
)
    # Шаблон для вопроса
    question = f"\n\nQuestion: {query}"

    # Добавляем к сообщению для chatGPT релевантные строки из базы знаний, пока не выйдем за допустимое число токенов
    for string in strings:
        next_article = f'\n\nWikipedia article section:\n"""\n{string}\n"""'
        if (num_tokens(message + next_article + question, model=model) > token_budget):
            break
        else:
            message += next_article
    return message + question


def ask(
    query: str, # пользовательский запрос
    df: pd.DataFrame = df, # DataFrame со столбцами text и embedding (база знаний)
    model: str = GPT_MODEL, # модель
    token_budget: int = 4096 - 500, # ограничение на число отсылаемых токенов в модель
    print_message: bool = False, # нужно ли выводить сообщение перед отправкой
) -> str:
    """Отвечает на вопрос, используя GPT и базу знаний."""
    # Формируем сообщение к chatGPT (функция выше)
    message = query_message(query, df, model=model, token_budget=token_budget)
    # Если параметр True, то выводим сообщение
    if print_message:
        print(message)
    messages = [
        {"role": "system", "content": "Ты отвечаешь на вопрос об Великой Отечественной войне"},
        {"role": "user", "content": message},
    ]
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0 # гиперпараметр степени случайности при генерации текста. Влияет на то, как модель выбирает следующее слово в последовательности.
    )
    response_message = response.choices[0].message.content
    return response_message

In [6]:
!pip install aiogram
!pip install nest_asyncio
import nest_asyncio
nest_asyncio.apply()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.4/698.4 kB 9.1 MB/s eta 0:00:00


In [21]:
import asyncio
import logging
from aiogram import Bot, Dispatcher, types
from aiogram.filters.command import Command
import os

logging.basicConfig(level=logging.INFO)

key = userdata.get('BOT_API_KEY')
os.environ["BOT_API_KEY"] = key
API_TOKEN = os.environ.get("BOT_API_KEY")

bot = Bot(token=API_TOKEN)
dp = Dispatcher()

@dp.message(Command("start"))
async def cmd_start(message: types.Message):
    await message.answer("Привет! Я дедушка Фронтовик! Я расскажу тебе множество различной информации про Великую Отечественную войну! Задай свой вопрос.\n\nДля получения дополнительной информации напиши \"/help\"")

@dp.message(Command("help"))
async def cmd_help(message: types.Message):
    await message.answer("Я отвечу на любой вопрос на тему Великой Отечественной войны.\n\nЯ помню информацию на 904 темы.\n\nКак ты можешь задать мне вопрос: \n\nКогда была великая отечественная война?\nКакие были общие потери во время войны?\nРасскажи про блокаду Ленинграда")

@dp.message()
async def cmd_ask(message: types.Message):
    thinking_msg = await message.answer("Вспоминаю...")
    answer = ask(message.text)
    await thinking_msg.edit_text(answer)

async def main():
    await dp.start_polling(bot)

if __name__ == "__main__":
    asyncio.run(main())

In [42]:
from scipy import spatial
from openai import OpenAI
import os
import re
import pandas as pd

def embedding_similarity(text1: str, text2: str, model="text-embedding-ada-002") -> float:
    try:
        resp = openai.embeddings.create(model=model, input=[text1, text2])
        emb1, emb2 = resp.data[0].embedding, resp.data[1].embedding
        return 1 - spatial.distance.cosine(emb1, emb2)
    except Exception as e:
        print(f"Embedding error: {e}")
        return 0.0

def extract_facts(text: str) -> dict:
    """Извлекает ключевые факты: даты, числа, ФИО, названия в кавычках"""
    facts = {}
    dates = re.findall(
        r'\b\d{1,2}[-\s\.](?:января|февраля|марта|апреля|мая|июня|июля|августа|сентября|октября|ноября|декабря)[-\s\.]?\d{4}\b|'
        r'\b\d{1,2}[\.\/]\d{1,2}[\.\/]\d{4}\b|'
        r'\b194[1-5]\b',
        text, re.IGNORECASE
    )
    facts["dates"] = list(set(dates))
    numbers = re.findall(r'\b([1-9][0-9]{2,})\b', text)
    facts["numbers"] = list(set(numbers))
    names = re.findall(r'\b[А-ЯЁ][а-яё]+(?:\s+[А-ЯЁ][а-яё]+){0,2}\b', text)
    facts["names"] = list(set(names))
    quotes = re.findall(r'[«"]([^»"]+)[»"]', text)
    facts["quoted"] = [q.strip() for q in quotes if len(q) > 3]

    return facts

def fact_overlap(pred: str, ref: str) -> float:
    fp, fr = extract_facts(pred), extract_facts(ref)
    overlap, total = 0, 0
    for key in ["dates", "numbers", "names", "quoted"]:
        if fp[key] or fr[key]:
            total += 1
            if set(fp[key]) & set(fr[key]):
                overlap += 1
    return overlap / total if total > 0 else 1.0

def is_context_relevant(retrieved: str, ground_truth_contexts: list[str]) -> bool:
    for gt_ctx in ground_truth_contexts:

        key_entities = set()
        key_entities.update(re.findall(r'\b[А-Я][а-яё]+(?:\s+[А-Я][а-яё]+){1,2}\b', gt_ctx))
        key_entities.update(re.findall(r'[«"]([^»"]+)[»"]', gt_ctx))
        key_entities.update(re.findall(r'\b194[1-5]\b', gt_ctx))
        retrieved_lower = retrieved.lower()
        for ent in key_entities:
            if ent.lower() in retrieved_lower:
                return True
    return False

def evaluate_retrieval(query: str, ground_truth_contexts: list[str], df: pd.DataFrame, top_k: int = 5) -> dict:
    retrieved_strings, _ = strings_ranked_by_relatedness(query, df, top_n=top_k)
    hits = [is_context_relevant(ctx, ground_truth_contexts) for ctx in retrieved_strings]
    hit_at_k = any(hits)
    rr = 1 / (hits.index(True) + 1) if True in hits else 0
    return {
        "retrieved_contexts": retrieved_strings[:3],
        "hit@k": hit_at_k,
        "rr": rr,
        "retrieval_correct": hit_at_k
    }

def evaluate_generation(
    query: str,
    ground_truth_answer: str,
    df_kb: pd.DataFrame,
    model="gpt-3.5-turbo",
    token_budget=4096 - 500
) -> dict:
    try:
        model_answer = ask(query=query, df=df_kb, model=model, token_budget=token_budget, print_message=False)
    except Exception as e:
        model_answer = f"[ERROR: {str(e)}]"

    emb_sim = embedding_similarity(model_answer, ground_truth_answer)
    fact_sim = fact_overlap(model_answer, ground_truth_answer)
    model_answer_clean = model_answer.lower().strip(" .,!?\"'«»")
    is_noncommittal = (model_answer_clean == "я не помню")
    generation_correct = (
        (not is_noncommittal) and
        ((emb_sim > 0.75) or (fact_sim == 1.0))
    )

    return {
        "model_answer": model_answer,
        "emb_similarity": emb_sim,
        "fact_similarity": fact_sim,
        "is_noncommittal": is_noncommittal,
        "generation_correct": generation_correct
    }

def evaluate_end_to_end(row: pd.Series, df_kb: pd.DataFrame) -> dict:
    ret = evaluate_retrieval(row["query"], row["relevant_contexts"], df_kb, top_k=5)
    gen = evaluate_generation(row["query"], row["ground_truth_answer"], df_kb)
    return {
        **ret,
        **gen,
        "e2e_correct": gen["generation_correct"],
        "difficulty": row["difficulty"]
    }

test_questions = [
    {
        "query": "Когда началась Великая Отечественная война?",
        "ground_truth_answer": "22 июня 1941 года",
        "relevant_contexts": ["22 июня 1941 года в 4 часа утра без объявления войны германские войска напали на СССР."],
        "difficulty": "easy"
    },
    {
        "query": "Когда была снята блокада Ленинграда?",
        "ground_truth_answer": "27 января 1944 года",
        "relevant_contexts": ["Блокада Ленинграда была полностью снята 27 января 1944 года в результате операции «Январь»."],
        "difficulty": "medium"
    },
    {
        "query": "Кто был Верховным Главнокомандующим в годы ВОВ?",
        "ground_truth_answer": "Иосиф Сталин",
        "relevant_contexts": ["6 августа 1941 года И. В. Сталин был назначен Верховным Главнокомандующим Вооружёнными Силами СССР."],
        "difficulty": "easy"
    },
    {
        "query": "Что такое операция «Багратион»?",
        "ground_truth_answer": "Советская стратегическая операция по освобождению Белоруссии летом 1944 года",
        "relevant_contexts": [
            "Операция «Багратион» — стратегическая наступательная операция Красной армии летом 1944 года, приведшая к уничтожению группы армий «Центр» и освобождению Белоруссии."
        ],
        "difficulty": "medium"
    },
    {
        "query": "Что такое «Дорога жизни»?",
        "ground_truth_answer": "Автозимник через Ладожское озеро для снабжения блокадного Ленинграда",
        "relevant_contexts": [
            "«Дорога жизни» — единственная транспортная артерия, связывавшая осаждённый Ленинград с остальной страной по льду Ладожского озера зимой 1941–1942 и 1942–1943 годов."
        ],
        "difficulty": "easy"
    },
    {
        "query": "Какая битва считается крупнейшей танковой битвой в истории?",
        "ground_truth_answer": "Курская битва (Прохоровское сражение)",
        "relevant_contexts": [
            "Прохоровское танковое сражение 12 июля 1943 года в ходе Курской битвы — крупнейшее в истории по количеству участвовавших танков."
        ],
        "difficulty": "medium"
    },
    {
        "query": "Сколько дней длилась блокада Ленинграда?",
        "ground_truth_answer": "872 дня",
        "relevant_contexts": [
            "Ленинград находился в блокаде с 8 сентября 1941 года по 27 января 1944 года — 872 дня."
        ],
        "difficulty": "medium"
    },
    {
        "query": "Какие были общие людские потери СССР в ВОВ?",
        "ground_truth_answer": "Около 27 миллионов человек",
        "relevant_contexts": [
            "По официальным данным, общие демографические потери СССР в Великой Отечественной войне составили около 26,6 миллиона человек."
        ],
        "difficulty": "medium"
    },
    {
        "query": "Почему германское наступление под Курском провалилось?",
        "ground_truth_answer": "Советское командование знало о планах операции «Цитадель» и подготовило глубокую оборону",
        "relevant_contexts": [
            "Советская разведка (в т.ч. «Люци» и «Колья») передала точные данные о сроках и направлениях операции «Цитадель», что позволило создать мощную оборону на Курской дуге."
        ],
        "difficulty": "hard"
    },
    {
        "query": "Какое обращение Молотова стало первым официальным сообщением о войне?",
        "ground_truth_answer": "Обращение по радио 22 июня 1941 года",
        "relevant_contexts": [
            "22 июня 1941 года в 12:00 по радио выступил В. М. Молотов с обращением «Наше дело правое — враг будет побеждён!»."
        ],
        "difficulty": "medium"
    }
]

test_df = pd.DataFrame(test_questions)

print("Запуск оценки RAG-системы (retrieval + generation + e2e)...\n")

results = []
for i, row in test_df.iterrows():
    print(f"{i+1}/{len(test_questions)} {row['query']}")
    try:
        res = evaluate_end_to_end(row, df_kb=df)
        results.append(res)
    except Exception as e:
        print(f"Ошибка: {e}")
        results.append({"error": str(e)})

results_df = pd.DataFrame(results)
test_with_results = pd.concat([test_df, results_df], axis=1)

print("\n" + "="*50)
print("ИТОГОВАЯ ОЦЕНКА")
print("="*50)
total = len(test_with_results)
hit5 = test_with_results['hit@k'].mean()
e2e = test_with_results['e2e_correct'].mean()

print(f"Всего вопросов: {total}")
print(f"Retrieval Hit@5:    {hit5:.1%}")
print(f"End-to-End Accuracy: {e2e:.1%}")

print("\nПримеры ответов:")
for i, row in test_with_results.head(10).iterrows():
    print(f"\n→ {row['query']}")
    print(f"   Модель: {row['model_answer'][:80]}{'...' if len(row['model_answer'])>80 else ''}")
    print(f"   Эталон: {row['ground_truth_answer']}")
    print(f"   E2E: {'Да' if row['e2e_correct'] else 'Нет'} | Hit@5: {'Да' if row['hit@k'] else 'Нет'}")


Запуск оценки RAG-системы (retrieval + generation + e2e)...

1/10 Когда началась Великая Отечественная война?
2/10 Когда была снята блокада Ленинграда?
3/10 Кто был Верховным Главнокомандующим в годы ВОВ?
4/10 Что такое операция «Багратион»?
5/10 Что такое «Дорога жизни»?
6/10 Какая битва считается крупнейшей танковой битвой в истории?
7/10 Сколько дней длилась блокада Ленинграда?
8/10 Какие были общие людские потери СССР в ВОВ?
9/10 Почему германское наступление под Курском провалилось?
10/10 Какое обращение Молотова стало первым официальным сообщением о войне?

ИТОГОВАЯ ОЦЕНКА
Всего вопросов: 10
Retrieval Hit@5:    100.0%
End-to-End Accuracy: 100.0%

Примеры ответов:

→ Когда началась Великая Отечественная война?
   Модель: 22 июня 1941 года.
   Эталон: 22 июня 1941 года
   E2E: Да | Hit@5: Да

→ Когда была снята блокада Ленинграда?
   Модель: Блокада Ленинграда была полностью снята 27 января 1944 года.
   Эталон: 27 января 1944 года
   E2E: Да | Hit@5: Да

→ Кто был Верховным Главно